In [4]:
import sys
print(sys.executable)


C:\Users\india\anaconda3\envs\mlops\python.exe


In [5]:
!conda info --envs
!where jupyter


# conda environments:
#
base                   C:\Users\india\anaconda3
mlops                * C:\Users\india\anaconda3\envs\mlops
redteam                C:\Users\india\anaconda3\envs\redteam
resai                  C:\Users\india\anaconda3\envs\resai

C:\Users\india\anaconda3\Scripts\jupyter.exe


In [6]:
import os

In [7]:
%cd C:\Users\india\Desktop\Jio_Institute\MLOps\Project\Nagendra\MLOPS

C:\Users\india\Desktop\Jio_Institute\MLOps\Project\Nagendra\MLOPS


C:\Users\india\AppData\Roaming\Python\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
print("Current working directory:", os.getcwd())


Current working directory: C:\Users\india\Desktop\Jio_Institute\MLOps\Project\Nagendra\MLOPS


In [9]:
import pandas as pd
import requests
from io import BytesIO
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

In [10]:
def load_parquet_from_github(url):
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful
    return pd.read_parquet(BytesIO(response.content))

In [11]:
# Define raw URLs for train, test, and prod datasets.
train_url = "https://raw.githubusercontent.com/Jupudy-Nagendra/MLOPS/main/Dataset/Parquet/Metro_Interstate_Traffic_Volume_train.parquet"
test_url  = "https://raw.githubusercontent.com/Jupudy-Nagendra/MLOPS/main/Dataset/Parquet/Metro_Interstate_Traffic_Volume_test.parquet"
prod_url  = "https://raw.githubusercontent.com/Jupudy-Nagendra/MLOPS/main/Dataset/Parquet/Metro_Interstate_Traffic_Volume_prod.parquet"


In [12]:
train_df = load_parquet_from_github(train_url)
test_df  = load_parquet_from_github(test_url)
prod_df  = load_parquet_from_github(prod_url)

In [13]:
train_df = train_df.sort_values(by="date_time")
test_df = test_df.sort_values(by="date_time")
prod_df = prod_df.sort_values(by="date_time")

In [24]:
train_df.columns

Index(['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main',
       'weather_description', 'date_time', 'traffic_volume'],
      dtype='object')

In [186]:
train_df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
19662,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
6094,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
15731,None,291.72,0.0,0.0,1,Clear,sky is clear,2012-10-02 14:00:00,5181
2174,None,294.14,0.0,0.0,20,Clouds,few clouds,2012-10-02 17:00:00,5791
28172,None,293.10,0.0,0.0,20,Clouds,few clouds,2012-10-02 18:00:00,4770


In [188]:
train_df.columns

Index(['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main',
       'weather_description', 'date_time', 'traffic_volume'],
      dtype='object')

In [28]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

In [30]:
class DateTimeExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, column="date_time", drop_original=True):
        self.column = column
        self.drop_original = drop_original
        
    def fit(self, X, y=None):
        # No fitting necessary for this transformer.
        return self
    
    def transform(self, X):
        X = X.copy()
        # Convert the column to datetime
        X[self.column] = pd.to_datetime(X[self.column])
        # Create new columns for year, month, day, and hour
        X['year'] = X[self.column].dt.year
        X['month'] = X[self.column].dt.month
        X['day'] = X[self.column].dt.day
        X['hour'] = X[self.column].dt.hour
        # Optionally drop the original date_time column
        if self.drop_original:
            X = X.drop(columns=[self.column])
        return X

In [45]:
# Define the feature columns.
numerical_cols = ['temp', 'rain_1h', 'snow_1h', 'clouds_all', 'year', 'month', 'day', 'hour']
categorical_cols = [ 'weather_main', 'weather_description']

In [37]:
datetime_pipeline = Pipeline(steps=[
    ('datetime_extractor', DateTimeExtractor(column="date_time", drop_original=True))
])

In [39]:
# Define a holiday transformer to convert the 'holiday' column to a binary feature.
# If the value is missing or "None", it becomes 0; otherwise 1.
holiday_transformer = FunctionTransformer(
    lambda x: ((~pd.isnull(x)) & (x != "None")).astype(int),
    validate=False
)

In [41]:
# Build the numerical transformer pipeline.
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Build the categorical transformer pipeline.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [47]:
# Combine all transformers in a ColumnTransformer.
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols),
    ('holiday', holiday_transformer, ['holiday'])
])

In [51]:
full_pipeline = Pipeline(steps=[
    ('datetime', datetime_pipeline),      # First extract datetime features
    ('preprocessor', preprocessor)          # Then process numerical and categorical features
    # You can add a model step here, e.g., ('model', SomeRegressor())
])

In [55]:
X_transformed = full_pipeline.fit_transform(train_df)

In [59]:
import numpy as np

In [61]:
X_transformed_dense = X_transformed if isinstance(X_transformed, np.ndarray) else X_transformed.toarray()


In [63]:
print("Transformed data shape:", X_transformed_dense.shape)
print("First row of transformed data:")
print(X_transformed_dense[0])

Transformed data shape: (28922, 58)
First row of transformed data:
[ 0.63140899 -0.13500797 -0.02556149  1.04359682 -1.86399345  1.0234557
 -1.56852123 -0.06092323  0.          1.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          1.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]


In [69]:
try:
    feature_names = full_pipeline.named_steps['preprocessor'].get_feature_names_out()
    print("Output Feature Names:")
    print(feature_names)
except Exception as e:
    print("Error retrieving feature names:", e)

# For demonstration, print the shape and first row of the transformed data.
print("\nTransformed data shape:", X_transformed_dense.shape)
print("First row of transformed data:")
print(X_transformed_dense[0])

Error retrieving feature names: Transformer holiday (type FunctionTransformer) does not provide get_feature_names_out.

Transformed data shape: (28922, 58)
First row of transformed data:
[ 0.63140899 -0.13500797 -0.02556149  1.04359682 -1.86399345  1.0234557
 -1.56852123 -0.06092323  0.          1.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          1.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        ]


In [71]:
# Build the full ML pipeline with a Linear Regression model.
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [73]:
# Prepare training data: separate features and target.
X_train = train_df.drop("traffic_volume", axis=1)
y_train = train_df["traffic_volume"]

In [75]:
# Fit the pipeline on the training data.
model_pipeline.fit(X_train, y_train)
print("ML Pipeline built and model trained successfully!")

ValueError: A given column is not a column of the dataframe

In [150]:
import sklearn.metrics as metrics

# Prepare test set features and labels.
X_test = test_df.drop("traffic_volume", axis=1)
y_test = test_df["traffic_volume"]

In [261]:
import numpy as np
import sklearn.metrics as metrics

# Get predictions from the model.
predictions = model_pipeline.predict(X_test)

# Calculate Mean Squared Error, then take the square root for RMSE.
mse = metrics.mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

# Calculate R-squared (R2 score).
r2 = metrics.r2_score(y_test, predictions)

print("Evaluation Metrics on Test Data:")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R2 Score): {r2:.2f}")


Evaluation Metrics on Test Data:
Root Mean Squared Error (RMSE): 5623.18
R-squared (R2 Score): -6.94


In [257]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, FunctionTransformer

In [259]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class HolidayBinaryTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Assumes X is a DataFrame with one column 'holiday'
        # Convert holiday values to binary: 0 if null or "None", else 1.
        return ((~pd.isnull(X)) & (X != "None")).astype(int)
    
    def get_feature_names_out(self, input_features=None):
        # Return a single feature name for the holiday column.
        return ['holiday']

# Example usage with ColumnTransformer:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Example feature columns (assuming X_train is your DataFrame)
numerical_cols = ['temp', 'rain_1h', 'snow_1h', 'clouds_all']
categorical_cols = ['weather_main', 'weather_description']

# Build numerical transformer pipeline.
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

# Build categorical transformer pipeline.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Use the custom HolidayBinaryTransformer for the holiday column.
holiday_transformer = HolidayBinaryTransformer()

# Combine all transformers.
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols),
    ('holiday', holiday_transformer, ['holiday'])
])

# Fit the preprocessor with training data (assuming X_train exists).
# For demonstration, let's assume X_train is defined:
# X_train = your DataFrame with all columns including 'holiday'
preprocessor.fit(X_train)

# Now, get the output feature names.
feature_names = preprocessor.get_feature_names_out()
print("Transformed Feature Names:")
print(feature_names)


Transformed Feature Names:
['num__temp' 'num__rain_1h' 'num__snow_1h' 'num__clouds_all'
 'cat__weather_main_Clear' 'cat__weather_main_Clouds'
 'cat__weather_main_Drizzle' 'cat__weather_main_Fog'
 'cat__weather_main_Haze' 'cat__weather_main_Mist'
 'cat__weather_main_Rain' 'cat__weather_main_Smoke'
 'cat__weather_main_Snow' 'cat__weather_main_Squall'
 'cat__weather_main_Thunderstorm' 'cat__weather_description_SQUALLS'
 'cat__weather_description_Sky is Clear'
 'cat__weather_description_broken clouds'
 'cat__weather_description_drizzle' 'cat__weather_description_few clouds'
 'cat__weather_description_fog' 'cat__weather_description_freezing rain'
 'cat__weather_description_haze'
 'cat__weather_description_heavy intensity drizzle'
 'cat__weather_description_heavy intensity rain'
 'cat__weather_description_heavy snow'
 'cat__weather_description_light intensity drizzle'
 'cat__weather_description_light intensity shower rain'
 'cat__weather_description_light rain'
 'cat__weather_description_lig

In [224]:
print("X_train columns:", X_train.columns.tolist())


X_train columns: ['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main', 'weather_description', 'date_time']


In [226]:
# Numerical features
num_out = numerical_transformer.fit_transform(X_train[numerical_cols])
print("Numerical output shape:", num_out.shape)

# Categorical features
cat_out = categorical_transformer.fit_transform(X_train[categorical_cols])
print("Categorical output shape:", cat_out.shape)

# Holiday feature (using the custom transformer)
holiday_out = holiday_transformer.fit_transform(X_train[['holiday']])
print("Holiday output shape:", holiday_out.shape)


Numerical output shape: (28922, 4)
Categorical output shape: (28922, 49)
Holiday output shape: (28922, 1)


In [228]:
import numpy as np

# Transform each subset
num_transformed = numerical_transformer.transform(X_train[numerical_cols])
cat_transformed = categorical_transformer.transform(X_train[categorical_cols])
holiday_transformed = holiday_transformer.transform(X_train[['holiday']])

# If the categorical output is a sparse matrix, convert it to a dense array
if hasattr(cat_transformed, "toarray"):
    cat_transformed = cat_transformed.toarray()

# Combine all outputs horizontally
combined = np.hstack([num_transformed, cat_transformed, holiday_transformed])
print("Combined transformed shape:", combined.shape)


Combined transformed shape: (28922, 54)


In [230]:
# For numerical, we can simply use the column names.
num_feature_names = numerical_cols

# For categorical, OneHotEncoder provides get_feature_names_out
cat_feature_names = categorical_transformer.named_steps['onehot'].get_feature_names_out(categorical_cols)

# For holiday, our custom transformer returns a fixed name.
holiday_feature_names = holiday_transformer.get_feature_names_out()

# Combine the feature names
all_feature_names = list(num_feature_names) + list(cat_feature_names) + list(holiday_feature_names)
print("Total number of feature names:", len(all_feature_names))


Total number of feature names: 54


In [232]:
print("Combined transformed shape:", combined.shape)  # Expect (n_samples, len(all_feature_names))


Combined transformed shape: (28922, 54)


In [240]:
type(combined)

numpy.ndarray

In [242]:
# Assuming 'combined' is your NumPy ndarray
print("First row of transformed data:")
print(combined[0])

First row of transformed data:
[ 0.63140899 -0.13500797 -0.02556149  1.04359682  0.          1.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          1.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]


In [244]:
import pandas as pd

# Assume 'combined' is your NumPy array from the transformed data,
# and 'all_feature_names' is a list of column names corresponding to each feature.
# For demonstration, let's print the first row.

row_df = pd.DataFrame([combined[0]], columns=all_feature_names)
print("First row with columns:")
print(row_df)


First row with columns:
       temp   rain_1h   snow_1h  clouds_all  weather_main_Clear  \
0  0.631409 -0.135008 -0.025561    1.043597                 0.0   

   weather_main_Clouds  weather_main_Drizzle  weather_main_Fog  \
0                  1.0                   0.0               0.0   

   weather_main_Haze  weather_main_Mist  ...  weather_description_smoke  \
0                0.0                0.0  ...                        0.0   

   weather_description_snow  weather_description_thunderstorm  \
0                       0.0                               0.0   

   weather_description_thunderstorm with drizzle  \
0                                            0.0   

   weather_description_thunderstorm with heavy rain  \
0                                               0.0   

   weather_description_thunderstorm with light drizzle  \
0                                                0.0     

   weather_description_thunderstorm with light rain  \
0                                     

In [246]:
import numpy as np

# Assuming 'combined' is your transformed NumPy array and the holiday column is the last column:
holiday_col = combined[:, -1]

# Use np.unique to count unique values and their frequencies.
unique, counts = np.unique(holiday_col, return_counts=True)

# Print the results.
print("Holiday column value counts:")
for value, count in zip(unique, counts):
    print(f"Value {int(value)}: {count}")


Holiday column value counts:
Value 0: 28879
Value 1: 43


In [266]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   -- ------------------------------------- 6.8/124.9 MB 38.1 MB/s eta 0:00:04
   --- ------------------------------------ 11.5/124.9 MB 30.1 MB/s eta 0:00:04
   ---- ----------------------------------- 14.7/124.9 MB 24.3 MB/s eta 0:00:05
   ----- ---------------------------------- 17.8/124.9 MB 21.6 MB/s eta 0:00:05
   ------ --------------------------------- 21.5/124.9 MB 20.9 MB/s eta 0:00:05
   -------- ------------------------------- 25.2/124.9 MB 20.7 MB/s eta 0:00:05
   -------- ------------------------------- 26.5/124.9 MB 18.4 MB/s eta 0:00:06
   --------- ------------------------------ 29.4/124.9 MB 17.9 MB/s eta 0:00:06
   --------- ------------------------------ 31.2/124.9 MB 16.9 MB/s eta 0:00:06
   ---------- ----------------------------- 33.3/124.9 MB 16.3 MB/s eta 0:00:06
   ----------- ---------------------------- 36.2/124.9 MB 15.9 MB/s eta 0:00:06
   ------------ --------------------------- 38.5/1